In [1]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoConfig, AutoModelForSequenceClassification
from transformers import CanineForSequenceClassification, CanineTokenizer, BertTokenizer
from torch.utils.data import Dataset
from datasets import load_metric
from sklearn.metrics import roc_auc_score
from scipy.special import softmax
from transformers import TrainingArguments
from transformers import Trainer
from sklearn.utils import resample
import torch.nn as nn

In [2]:
def compute_metrics(eval_pred):
    precision = load_metric("precision")
    recall = load_metric("recall")
    f1 = load_metric("f1")
    acc = load_metric("accuracy")
    mcc = load_metric("matthews_correlation")
    #auc = load_metric("auc")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision.compute(predictions=predictions, average = "macro", references=labels)["precision"]
    recall = recall.compute(predictions=predictions, average = "macro", references=labels)["recall"]
    f1 = f1.compute(predictions=predictions, average = "macro", references=labels)["f1"]
    acc = acc.compute(predictions=predictions, references=labels)["accuracy"]
    mcc = mcc.compute(predictions=predictions, references=labels)["matthews_correlation"]
    auc = roc_auc_score(labels, softmax(logits, axis=1), multi_class='ovo', average='macro')
    return {"precision": precision, "recall": recall, "acc": acc, "mcc": mcc, "f1": f1, "auc":auc}

In [3]:
class MalwareDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [4]:
class RandomTransformerClassifier:
    def __init__(self, trainingset, validationset, testset, num_classes, epochs=10, batch_size=8, model_name='google/canine-s', max_sequence_length=2048):
        self.trainingset = trainingset
        self.validationset = validationset
        self.testset = testset
        self.num_classes = num_classes
        self.model_name = model_name
        if 'google/canine' == model_name.split('-')[0]:
            self.tokenizer = CanineTokenizer.from_pretrained(self.model_name)
        else:
            self.tokenizer = BertTokenizer.from_pretrained(self.model_name)
            
        self.tokenizer.model_max_length = max_sequence_length
        self.base_model = AutoModelForSequenceClassification.from_pretrained(self.model_name, num_labels=num_classes)
        self.training_args = TrainingArguments(
        output_dir='./results',          # output directory
        #do_train=True,
        do_eval=True,
        evaluation_strategy='epoch',
        save_strategy='epoch',
        load_best_model_at_end=True,
        metric_for_best_model='eval_auc',
        greater_is_better=True,
        fp16=True,
        num_train_epochs=epochs,              # total number of training epochs
        per_device_train_batch_size=batch_size,  # batch size per device during training
        per_device_eval_batch_size=batch_size,   # batch size for evaluation
        lr_scheduler_type='cosine',
        warmup_steps=500,                # number of warmup steps for learning rate scheduler
        weight_decay=0.01,               # strength of weight decay
        dataloader_num_workers=16,
        #logging_dir='./logs',            # directory for storing logs
        #logging_steps=10,
    )
        
    def fit(self, n_estimators=2):
        self.validation_preds = []
        self.test_preds = []
        for i in range(n_estimators):
            #bagging_trainset = self.trainingset.sample(frac=1, replace=True)
            bagging_trainset = resample(self.trainingset, replace=True, stratify=self.trainingset['class'])
            train_encodings = self.tokenizer(self.get_list_strs(bagging_trainset.api), padding="max_length", truncation=True, return_tensors="pt")
            val_encodings = self.tokenizer(self.get_list_strs(self.validationset.api), padding="max_length", truncation=True, return_tensors="pt")
            test_encodings = self.tokenizer(self.get_list_strs(self.testset.api), padding="max_length", truncation=True, return_tensors="pt")
            
            trainset = MalwareDataset(train_encodings, bagging_trainset['class'].values)
            valset = MalwareDataset(val_encodings, self.validationset['class'].values)
            testset = MalwareDataset(test_encodings, self.testset['class'].values)
            
            trainer = WeightedLossTrainer(
                model=self.base_model, args=self.training_args, train_dataset=trainset, eval_dataset=valset,
                compute_metrics=compute_metrics
            )
            
            trainer.train()
            
            val_preds = trainer.predict(valset)
            test_preds = trainer.predict(testset)
            self.validation_preds.append(val_preds)
            self.test_preds.append(test_preds)
            del bagging_trainset
            del train_encodings
            del val_encodings
            del test_encodings
            del trainset
            del valset
            del testset
            del trainer
            torch.cuda.empty_cache()
            print(f'{i + 1}. estimator is done....')
    def get_preds(self):
        return self.validation_preds, self.test_preds
    
    def get_metrics(self):
        #np.argmax((softmax(val[0].predictions, axis=1) + softmax(val[1].predictions, axis=1))/2, axis=1)
        val_preds = np.zeros_like(self.validation_preds[0].predictions)
        for val in self.validation_preds:
            val_preds += softmax(val.predictions, axis=1)
        
        val_logits = val_preds / len(self.validation_preds)
        val_preds = np.argmax(val_preds / len(self.validation_preds), axis=1)
        
        test_preds = np.zeros_like(self.test_preds[0].predictions)
        for test in self.test_preds:
            test_preds += softmax(test.predictions, axis=1)
        
        test_logits = test_preds / len(self.test_preds)
        test_preds = np.argmax(test_preds / len(self.test_preds), axis=1)
        precision = load_metric("precision")
        recall = load_metric("recall")
        f1 = load_metric("f1")
        acc = load_metric("accuracy")
        mcc = load_metric("matthews_correlation")
    
        val_precision = precision.compute(predictions=val_preds, average = "macro", references=self.validationset["class"].values)["precision"]
        val_recall = recall.compute(predictions=val_preds, average = "macro", references=self.validationset["class"].values)["recall"]
        val_f1 = f1.compute(predictions=val_preds, average = "macro", references=self.validationset["class"].values)["f1"]
        val_acc = acc.compute(predictions=val_preds, references=self.validationset["class"].values)["accuracy"]
        val_mcc = mcc.compute(predictions=val_preds, references=self.validationset["class"].values)["matthews_correlation"]
        val_auc = roc_auc_score(self.validationset["class"].values, softmax(val_logits, axis=1), multi_class='ovo', average='macro')
        
        test_precision = precision.compute(predictions=test_preds, average = "macro", references=self.testset["class"].values)["precision"]
        test_recall = recall.compute(predictions=test_preds, average = "macro", references=self.testset["class"].values)["recall"]
        test_f1 = f1.compute(predictions=test_preds, average = "macro", references=self.testset["class"].values)["f1"]
        test_acc = acc.compute(predictions=test_preds, references=self.testset["class"].values)["accuracy"]
        test_mcc = mcc.compute(predictions=test_preds, references=self.testset["class"].values)["matthews_correlation"]
        test_auc = roc_auc_score(self.testset["class"].values, softmax(test_logits, axis=1), multi_class='ovo', average='macro')

        return {"val_precision": val_precision, "val_recall": val_recall, "val_acc": val_acc, "val_mcc": val_mcc, "val_f1": val_f1, "val_auc":val_auc}, {"test_precision": test_precision, "test_recall": test_recall, "test_acc": test_acc, "test_mcc": test_mcc, "test_f1": test_f1, "test_auc":test_auc}
        

        
    def get_list_strs(self, df):
        lst_str = []
        for i in range(len(df)):
            str_ = df.values[i]
            lst_str.append(str_)
        return lst_str
    

In [5]:
malware_calls_df = pd.read_csv("../datasets/GraphApiNames.csv")

In [6]:
malware_calls_df.columns = ["api", "class"]

In [7]:
malware_calls_df.head()

,api,class
0,NtClose NtOpenKey NtQueryValueKey NtClose NtOp...,Downloader
1,GetSystemTimeAsFileTime NtAllocateVirtualMemor...,Downloader
2,SetErrorMode OleInitialize LdrGetDllHandle Ldr...,Downloader
3,SetErrorMode OleInitialize LdrGetDllHandle Ldr...,Downloader
4,GetSystemTimeAsFileTime NtAllocateVirtualMemor...,Downloader


In [8]:
malware_calls_df["class"].value_counts()

Trojan        31979
Adware         5444
Downloader     1948
Ransomware      404
Agent           220
Riskware        216
Backdoor        135
Dropper         118
Virus           102
Name: class, dtype: int64

In [9]:
malware_calls_df = malware_calls_df[malware_calls_df["class"] != "Undefined"]

In [10]:
CAT2IDX = {
    'Trojan': 0,
    'Adware': 1,
    'Downloader': 2,
    'Ransomware': 3,
    'Agent': 4,
    'Riskware': 5,
    'Backdoor': 6,
    'Dropper': 7,
    'Virus': 8
}

IDX2CAT = {
    0:'Trojan',
    1:'Adware',
    2:'Downloader',
    3:'Ransomware',
    4:'Agent',
    5:'Riskware',
    6:'Backdoor',
    7:'Dropper',
    8:'Virus'   
}

In [11]:
malware_calls_df['class'] = malware_calls_df['class'].map(lambda x: CAT2IDX[x])

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(malware_calls_df,
test_size=0.2, random_state=75, stratify = malware_calls_df['class'])

In [13]:
X_train.head()

,api,class
32871,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0
37975,NtAllocateVirtualMemory SetErrorMode LoadStrin...,0
2269,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,1
8251,NtDuplicateObject LdrGetDllHandle LdrGetProced...,5
30750,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0


In [14]:
X_train, X_val = train_test_split(X_train, test_size=0.2, random_state=75, stratify = X_train['class'])

In [15]:
X_train.head()

,api,class
34919,GetSystemTimeAsFileTime GetSystemInfo NtCreate...,0
32251,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0
39529,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0
35779,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0
16297,GetSystemTimeAsFileTime LdrLoadDll LdrGetProce...,0


In [16]:
class_weights = (1 - (malware_calls_df['class'].value_counts().sort_index() / len(malware_calls_df))).values
class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights

tensor([0.2117, 0.8658, 0.9520, 0.9900, 0.9946, 0.9947, 0.9967, 0.9971, 0.9975],
       device='cuda:0')

In [17]:
from transformers import Trainer
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        outputs = model(**inputs)
        logits = outputs.get("logits")
        labels = inputs.get("labels")
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_func(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [18]:
classifier = RandomTransformerClassifier(X_train, X_val, X_test, num_classes=9, 
                                         epochs=20, batch_size=16, max_sequence_length=256) 
                                         #model_name = "bert-base-cased")
                                         
                                        

Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Some weights of CanineForSequenceClassification were not initialized from the model checkpoint at google/canine-s and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
classifier.fit(n_estimators=2)

Using amp fp16 backend
***** Running training *****
  Num examples = 25961
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 16240
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construc

Epoch,Training Loss,Validation Loss,Precision,Recall,Acc,Mcc,F1,Auc
1,1.142900,0.913344,0.332996,0.317800,0.849484,0.543858,0.322848,0.712092
2,0.838700,0.828855,0.338979,0.323400,0.854876,0.559974,0.328908,0.791609
3,0.807300,0.859531,0.331535,0.324883,0.846403,0.525956,0.323682,0.758102
4,0.763800,0.806887,0.421527,0.363689,0.849792,0.553845,0.371967,0.779911
5,0.772900,0.822006,0.427269,0.365184,0.844708,0.543453,0.342339,0.785783
6,0.754700,0.829104,0.384056,0.341528,0.840240,0.538730,0.349359,0.783549
7,0.740200,0.815650,0.377638,0.375344,0.849638,0.555806,0.374757,0.804333
8,0.733900,0.795936,0.497856,0.388075,0.852873,0.564969,0.411063,0.804346
9,0.714000,0.808915,0.493319,0.397549,0.810815,0.500507,0.413582,0.807342
10,0.696200,0.816007,0.441617,0.396021,0.849022,0.559135,0.407357,0.803810


***** Running Evaluation *****
  Num examples = 6491
  Batch size = 32
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/

/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserW

/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserW

...

/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserW

/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 6491
  Batch size = 32
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/

/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserW

***** Running Prediction *****
  Num examples = 8114
  Batch size = 32
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  import sys
/home/nova/anaconda3/envs/maltransform/lib/python3.7/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


2. estimator is done....


In [20]:
val, test = classifier.get_metrics()

In [21]:
val

{'val_precision': 0.43842148358666283,
 'val_recall': 0.42447677558703084,
 'val_acc': 0.8254506239408411,
 'val_mcc': 0.5235035905531275,
 'val_f1': 0.4238843079877546,
 'val_auc': 0.811729091809667}

In [22]:
test

{'test_precision': 0.44505920322185666,
 'test_recall': 0.43491572216126323,
 'test_acc': 0.8238846438254868,
 'test_mcc': 0.5207811619036313,
 'test_f1': 0.4309990038797732,
 'test_auc': 0.796821295956517}